In [2]:
// Breakup sources into multiple files
var fs = require('fs');
var _init = fs.readFileSync('init.js').toString();
eval(_init);
setupD3();

function isEmpty(arrOrObj) {
	if(!arrOrObj) return true;
	if(isArray(arrOrObj) && arrOrObj.length === 0) return true;
	if(typeof arrOrObj === 'object' && 
			Object.keys(arrOrObj).length === 0 ) return true;
	return false;
}

D3 - loader


In [3]:

function strip(name) {
    let ix = name.indexOf('(');
    if(ix >=0 ) return name.substr(0,ix);
    return name;
}

function createDir(path, refData) {
    if(fs.existsSync(path)) throw new Error("dir "+path+" exists");
    console.log(`fs.mkdirSync(${path});`); 
    fs.mkdirSync(path);
}

function isFile(path) {
    return fs.existsSync(path+'.js') || fs.existsSync(path)
}

function mapper(dict) {
    return l => dict[l] || l;
}

function allButLast(path) {
    let p = path.split('/');
    if(p.length > 1) return p.slice(0, p.length-1).join('/');
    return path;
}

function rmdir(path) {
    try {
        path = path || './';
        let files = fs.readdirSync(path);
        files.forEach( f => (console.log(f), fs.unlinkSync(`${path}/${f}`)));
    } catch(e) {
        console.log(e);
    }
    try {
        console.log("rmdir ", path);
        fs.rmdirSync(path)
    }catch(e) {
        console.log(e);
    }
}

undefined

In [8]:
function processDir(dir) {
    var files = readAll(dir);
    console.log(files);
    files.forEach(f => processFile(f,true));  
}


function readAll(dir, matchPath = /\.js\.tp$/) {
    let list =fs.readdirSync(dir).filter(n => n.match(matchPath)).filter(name => name[0] === '_');
    let files = list.map(name => name.replace(matchPath,'').substr(1));
    let usableFiles = files.filter(name =>!fs.existsSync(`$dir/name`));
    let existing = files.filter(name =>fs.existsSync(`$dir/name`));
    if(existing && existing.length > 0) console.log("The following directories alreaddy exist", existing)
    return files.map(file => ({dir, file, getData: getData}));
}

function getData(ext=".js.tp") {
    let {dir, file} = this;
    let str = fs.readFileSync(`${dir}/_${file}${ext}`, 'utf8').replace(/\r/g,'');
    
    let [first, ...blocks] = str.split(/\/\/@@?/);
    this.first = first;
    this.blocks = blocks;
    this.getImports = _getImports;
    return this;
}

function _getImports() {
    let {dir, file, first, blocks} = this;
    this.imports = first.split("\n").filter(line => line.startsWith('import ') );
    this.getExports = _getExports;
    return this;
}

function _getExports() {
    let {blocks} = this;
    //console.log(blocks.length);
    this.exports = blocks.map(_getExportsOfBlock);
    return this;
}

function _getExportsOfBlock(block) {
    let lines = block.split("\n");
    let exps = lines.filter(l => l.match(/export/));
    let exports = exps.map(l => l.split(/\s+/));
    let defaults = exports.filter(l => l[1]=== 'default').map(([_,__,type,name])=> ({type,name: strip(name)}));
    exports = exports.filter(l => l[1] !== 'default').map(([_,type,name])=> ({type,name: strip(name)}));
    
    if(defaults.length > 1) throw new Error("too name defaaults"+ defaults.toString());
    return {block, exports, defaults};
}


function getName(dir,anExp) {
    let {defaults, exports} = anExp;
    let name = '';
    let isDefault = !isEmpty(defaults)
    if( isDefault ) name = defaults[0].name;
    else name = exports[0].name;
    let path = dir+"/"+name+'.js';
    
    return [path, isDefault, name, exports.map(e => e.name)];
}


function transform(dir,inports) {
    let [parts,file] = inports.replace(/import(.*)from\s['"]([a-zA-z0-9* ._/-]+)(['"].*)/,"$1\t$2")
                    .split('\t');
    let prefix = ['.', '../'].filter(pre => isFile(dir+'/'+pre+file));
    if(prefix.length === 0) return [file,file];
    return [file, prefix[0]+file];
}   

function writeBlock(dir, aB, fileData ) {
    let [path,dflt, funcName, allExports] = fileData;
    console.log("Cretae file", path);
    let lines = aB.block.split(/\r?\n/)
        .map(l => l.replace(/\/\/##\s*/g, '')
                   .replace(/^@\s+\n/,'') );
    let imports = lines.filter(l => l.match(/import\s/));
        

    let pth = allButLast(path)
    let newImports = imports.map(l => {
        let [old,nw] = transform(pth, l );
        //console.log({pth, old,nw})
        return [l,l.replace(old,nw)]
    }).reduce((o,[k,v]) => (o[k]=v,o),{})
    newImports['@'] = '//'+path
    let data = lines.map(mapper(newImports)).join('\n');
    data = data
        .replace(/export\s+default\s+const\s+[a-zA-Z0-9_$]+\s+=/g, "export default ")
        //.replace(/export\s+const\s+[a-zA-Z0-9_$]+\s+=/g, "export default ")
    
    console.log(data);
    //console.log(block);
//    fs.writeFileSync(path, data);
    return fileData;
}



function processFile(aFile,toDelDir) {
   let fdat =  aFile.getData().getImports().getExports();
    console.log(Object.keys(fdat));
    console.log(fdat.file, 'dir',fdat.dir);
    let dir = fdat.dir+'/'+fdat.file;
    if(toDelDir) rmdir(dir);
    createDir(dir);
    let funcs = fdat.exports.map(e => writeBlock(dir, e,getName(dir,e)) );
    console.log(writeIndex(dir, funcs));
}

function writeIndex(dir, funcs) {
    let src = "";
    let other = 0;
    let path = funcs.length >0?funcs[0][0]:undefined;
    for(let i=0; i<funcs.length; i++) {
        let [d, isDefault, func, exports] = funcs[i];
        if(isDefault) {
          src += "import _"+func+" from './"+func+"';\n";
          src += "export const "+func+" = _"+func+";\n\n"
        } else src += "export * from './"+func+"';\n";
    }

    if(path) {
        let index = allButLast(path)+"/index.js";
        //fs.writeFileSync(index, src);
        return {index, src};
    }
    return undefined;
}


/*
    let block = aB.block.replace(/\/\/##/g, '').replace(/^@\n/,'')
                .replace(/"/g, "'")
                .replace(/'\.\/haveFrame/, "'../haveFrame")                
                .replace(/\.\/frame/, '../frame')
                .replace(/\.\.\/string\//g, '../../string/')
                .replace(/\.\.\/utils\//, '../../utils/')
                .replace(/\.\.\/array\//, '../../array /')
                ;
*/



undefined

In [9]:

processDir("../src/frame");
// processDir("../src/array");
// processDir("../src/string");
// processDir("../src/dict");
// processDir("../src/utils");

// var files = readAll();
// rmdir("../src/frame/frame-utils")
// //fs.rmdirSync("../src/frame/frame-utils")
// console.log(files.map(e => e.file))
// //console.log(files[0].getData().getImports().getExports().exports.map(({defaults},i) =>( defaults)));
// files.forEach(f => processFile(f,true));


[
  {
    dir: '../src/frame',
    file: 'frame-utils',
    getData: [Function: getData]
  },
  {
    dir: '../src/frame',
    file: 'groupby-utils',
    getData: [Function: getData]
  },
  {
    dir: '../src/frame',
    file: 'join-utils',
    getData: [Function: getData]
  }
]
[
  'dir',     'file',
  'getData', 'first',
  'blocks',  'getImports',
  'imports', 'getExports',
  'exports'
]
frame-utils dir ../src/frame
frameCount.js
frameFromBuffer.js
frameSum.js
frameToString.js
frameWithIndex.js
index.js
transpose.js
unionFrame.js
rmdir  ../src/frame/frame-utils
fs.mkdirSync(../src/frame/frame-utils);
Cretae file ../src/frame/frame-utils/frameFromBuffer.js

import {Frame} from '../frame';
import {dataSplit} from '../../string/csv';
/**
 * default splitter is pipe seperated values
 * @param  {[type]} buffer    [description]
 * @param  {[type]} frameName [description]
 * @param  {[type]} splitter  [description]
 * @return {[type]}           [description]
 */
export default function fram

[
  'dir',     'file',
  'getData', 'first',
  'blocks',  'getImports',
  'imports', 'getExports',
  'exports'
]
groupby-utils dir ../src/frame
accStr.js
gb.js
gbCount.js
gbMax.js
gbMean.js
gbMerge.js
gbMergeU.js
gbMin.js
gbStdDiv.js
gbSum.js
index.js
rmdir  ../src/frame/groupby-utils
fs.mkdirSync(../src/frame/groupby-utils);
Cretae file ../src/frame/groupby-utils/gbSum.js

import {toNumber} from '../../utils/sort-helper';
/**
 * [gbSum description]
 * @param  {[type]} name    [description]
 * @param  {[type]} newName [description]
 * @return {[type]}         [description]
 */
export default function gbSum(name,newName) {
	newName = newName || name;
	function sum(action,accum,count, val) {
		if( accum === undefined ) accum = 0;
				
		if( action !==1 ) return  [accum,count];
		else {
			let v = toNumber(val);
			return  v===undefined || isNaN(v) ?[accum,count]:[accum+v,count+1];
		}
	}
	return [sum,name,[()=>0,0],newName];
}


Cretae file ../src/frame/groupby-utils/gbCount.js

import i

joinOp.js
leftJoin.js
localGroupBy.js
matchIt.js
outerJoin.js
project.js
reverseProjectList1.js
rmdir  ../src/frame/join-utils
fs.mkdirSync(../src/frame/join-utils);
Cretae file ../src/frame/join-utils/EMPTY_KEY.js

export default  '@#$-EMPTY-$#@';


Cretae file ../src/frame/join-utils/changeNameTo.js

/**
 * changeNameTo takes a change name description array of two elements containing the old names and the new names
 * e.g oldName=newName 'a name change descriptor' where '=' is the seperator
 * return an array of 2 strings [oldName, newName]
 * @param  {string} name old name to new name descriptor, 'OLD=NEW'
 * @param  {string} sep  string that seperates the old name from the new
 * @return {[oldName, newName]}      and ar
 */
export default function changeNameTo(name, sep) {
	 let res = name.split(sep);
	 if( res.length === 1) res.push(name);
	 return res;
}


Cretae file ../src/frame/join-utils/colName.js

import changeNameTo from './changeNameTo';
/**
 * colName(['1.DX=DX1', '2.DX=

Cretae file ../src/frame/join-utils/getDict.js

import EMPTY_KEY from './EMPTY_KEY';
import {EMPTY_ARRAY} from '../../utils/constants';
export default function getDict(dict, key, noEmpty) { /*local*/
	if(noEmpty && !key) return EMPTY_ARRAY;
	key = key || EMPTY_KEY;
	return dict[key] || EMPTY_ARRAY;
}


Cretae file ../src/frame/join-utils/innerJoin.js

import haveFrame from '../haveFrame';
import colName from './colName';
import localGroupBy from './localGroupBy';
import changeNameTo from './changeNameTo';
import joinOp from './joinOp';
import project from './project';
import getDict from './getDict';

/**
 * innerJoin works like an SQL inner join, not quite as flexible, 
 *     select a as A, b, frame1.x from frame1, frame2
 *         where frame1.colName1 = frame2.colName2 and {filter} 
 *
 * 
 * @param  {Frame} frame1      [description]
 * @param  {Frame} frame2      [description]
 * @param  {Array<string>} projectList Array of column name mapping example ""
 * @param  {String} joinO

Cretae file ../src/frame/join-utils/matchIt.js
/*local*/ 
/**
 * leftJoin on one row from left and arr from right
 * @param  {[type]} res
 * @param  {[type]} row1
 * @param  {[type]} arr
 * @param  {[type]} cmp
 * @param  {[type]} tr
 * @return {[[row-element]} 
 */
export default function matchIt(res,row1,arr, cmp, tr) {
	let len,  cnt =0;
	if( !arr || arr.length === 0 ) len = 0;
	else len = arr.length;
	let transform = tr[0];
	
	for(let i=0; i< len; i++) {
		let row2 = arr[i];
		if(cmp(row1,row2)) {
			res.push(transform(row1,row2));
			cnt++;
		}
	}
	if( cnt === 0 ) {
		transform = tr[1];
		res.push(transform(row1,[]));
	}
	return res;
}


Cretae file ../src/frame/join-utils/outerJoin.js

import haveFrame from '../haveFrame';
import colName from './colName';
import getTransform from './getTransform';
import localGroupBy from './localGroupBy';
import changeNameTo from './changeNameTo';
import joinOp from './joinOp';
import project from './project';
import getDict from './getDict';
im

undefined

true

In [98]:
transform("../src/frame/frame-utils","import a, {aaa,bbb,cc} from \"../array/arrEQ\"  ");


'../../array/arrEQ'

In [99]:
transform("../src/frame/frame-utils","import a, {aaa,bbb,cc} from \"./frame\"  ");

'../frame'

undefined